In [ ]:
import numpy as np 
import pandas as pd 

def converterTempo(data):
    if(len(data) == 10):
        minutes = float(data[0:2])
        seconds = float(data[3:9])
        minutes = minutes*60
        
    if(len(data) == 9):
        minutes = float(data[0:1])
        seconds = float(data[2:8])
        minutes = minutes*60
    
    if(len(data) == 8):
        minutes = float(data[0:1])
        seconds = float(data[2:7])
        minutes = minutes*60
    
    return minutes + seconds

malhas = [1000, 2000, 3000, 4000, 5000]
flags = [0, 1, 2, 3]

temposFlagDevice = []
melhoresTemposPorMalha = []

eficienciasFlagDevice = []
melhoresEficienciasPorMalha = []

speedupsFlagDevice = []
melhoresSpeedupsPorMalha = []

# tab = '\t'

# dic = {"cores": malhas,
#        "tempo": temposNoFlag,
#        "eficiencia": eficienciasNoFlag,
#        "speedup": speedupsNoFlag}

# dataFrame = pd.DataFrame(dic)
# dataFrame.to_csv("dados.dat", index=False, sep=tab)


In [ ]:
temposNoFlag = []
eficienciasNoFlag = []
speedupsNoFlag = []

for malha in malhas:
    file = pd.read_csv(f'{malha}/noFlag.out', header = None, sep='\s+')
    
    if ',' in file[1][0]:
        tempo = file[1][0].replace(',', '.')
    else:
        tempo = file[1][0]
    
    tempo = converterTempo(tempo)
    temposNoFlag.append(tempo)
    
for i in range(np.size(malhas)):
    eficienciasNoFlag.append((1 - (temposNoFlag[i]/temposNoFlag[0]))*100)
    
for i in range(np.size(malhas)):
    speedupsNoFlag.append((temposNoFlag[0]/temposNoFlag[i]))
       
print(temposNoFlag)
print(eficienciasNoFlag)
print(speedupsNoFlag)

In [ ]:
temposFlagFast = []
eficienciasFlagFast = []
speedupsFlagFast = []
aux = []

for malha in malhas:
    for flag in flags:
        file = pd.read_csv(f'{malha}/noFlag.out', header = None, sep='\s+')
        
        if ',' in file[1][0]:
            tempo = file[1][0].replace(',', '.')
        else:
            tempo = file[1][0]
    
        tempo = converterTempo(tempo)
        aux.append(tempo)
    
    temposFlagFast.append(aux)
    aux = []


for i in range(np.size(malhas)):
    for j in range(np.size(flags)):
        aux.append((1 - (temposFlagFast[i][j]/temposFlagFast[i][0]))*100)
        
    eficienciasFlagFast.append(aux)
    aux = []
    
for i in range(np.size(malhas)):
    for j in range(np.size(flags)):
        aux.append((temposFlagFast[i][0]/temposFlagFast[i][j]))
        
    speedupsFlagFast.append(aux)
    aux = []

print(temposFlagFast)
print(eficienciasFlagFast)
print(speedupsFlagFast)        

In [ ]:
temposFlagHost = []
eficienciasFlagHost = []
speedupsFlagHost = []
aux = []

for malha in malhas:
    for flag in flags:
        file = pd.read_csv(f'{malha}/noFlag.out', header = None, sep='\s+')
        
        if ',' in file[1][0]:
            tempo = file[1][0].replace(',', '.')
        else:
            tempo = file[1][0]
    
        tempo = converterTempo(tempo)
        aux.append(tempo)
    
    temposFlagHost.append(aux)
    aux = []


for i in range(np.size(malhas)):
    for j in range(np.size(flags)):
        aux.append((1 - (temposFlagHost[i][j]/temposFlagHost[i][0]))*100)
        
    eficienciasFlagHost.append(aux)
    aux = []
    
for i in range(np.size(malhas)):
    for j in range(np.size(flags)):
        aux.append((temposFlagHost[i][0]/temposFlagHost[i][j]))
        
    speedupsFlagHost.append(aux)
    aux = []

print(temposFlagHost)
print(eficienciasFlagHost)
print(speedupsFlagHost)        

In [ ]:
temposFlagDevice = []
eficienciasFlagDevice = []
speedupsFlagDevice = []
aux = []

for malha in malhas:
    for flag in flags:
        file = pd.read_csv(f'{malha}/noFlag.out', header = None, sep='\s+')
        
        if ',' in file[1][0]:
            tempo = file[1][0].replace(',', '.')
        else:
            tempo = file[1][0]
    
        tempo = converterTempo(tempo)
        aux.append(tempo)
    
    temposFlagDevice.append(aux)
    aux = []


for i in range(np.size(malhas)):
    for j in range(np.size(flags)):
        aux.append((1 - (temposFlagDevice[i][j]/temposFlagDevice[i][0]))*100)
        
    eficienciasFlagDevice.append(aux)
    aux = []
    
for i in range(np.size(malhas)):
    for j in range(np.size(flags)):
        aux.append((temposFlagDevice[i][0]/temposFlagDevice[i][j]))
        
    speedupsFlagDevice.append(aux)
    aux = []

print(temposFlagDevice)
print(eficienciasFlagDevice)
print(speedupsFlagDevice)        